In [18]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import pdb
from PIL import ImageDraw, ImageFont
from matplotlib import patches, patheffects
import pandas as pd
import matplotlib.pyplot as plt
import pydicom
import numpy as np
from pathlib import Path
from glob import glob
from matplotlib.patches import Rectangle
import png
from tqdm import tqdm_notebook as tqdm
import os

In [8]:
dp = Path('../../datasets/RSNA_PDC/')

DICOMS_TEST = dp/'stage_1_test_images'

PNGS_TEST = dp/'test1_png'
PNGS_TEST.mkdir(exist_ok=True)

In [7]:
def from_dicom_to_png(dicom_path, png_path):
    ds = pydicom.dcmread(dicom_path)
    shape = ds.pixel_array.shape
    # Convert to float to avoid overflow or underflow losses.
    image_2d = ds.pixel_array.astype(float)
    # Rescaling grey scale between 0-255
    image_2d_scaled = (np.maximum(image_2d,0) / image_2d.max()) * 255.0
    # Convert to uint
    image_2d_scaled = np.uint8(image_2d_scaled)
    # Write the PNG file
    with open(png_path, 'wb') as png_file:
        w = png.Writer(shape[1], shape[0], greyscale=True)
        w.write(png_file, image_2d_scaled)

In [13]:
dcms = list(DICOMS_TEST.iterdir())
dcms[0:5]

[PosixPath('../../datasets/RSNA_PDC/stage_1_test_images/0f45a78f-d20b-4370-b0c9-00da4e9a8a05.dcm'),
 PosixPath('../../datasets/RSNA_PDC/stage_1_test_images/2ed57b2d-a30f-4728-a2f8-76bb825a813c.dcm'),
 PosixPath('../../datasets/RSNA_PDC/stage_1_test_images/13990031-9dbf-4e03-9505-0f9120bdb166.dcm'),
 PosixPath('../../datasets/RSNA_PDC/stage_1_test_images/1a2dc891-713d-4253-ad19-a1e7910ee2eb.dcm'),
 PosixPath('../../datasets/RSNA_PDC/stage_1_test_images/0d8c48b1-8f7a-46c6-b63d-056402ed5b52.dcm')]

In [14]:
len(dcms)

1000

In [17]:
p = dcms[0]
p

PosixPath('../../datasets/RSNA_PDC/stage_1_test_images/0f45a78f-d20b-4370-b0c9-00da4e9a8a05.dcm')

In [21]:
b = os.path.basename(p)

In [23]:
os.path.splitext(b)[0]

'0f45a78f-d20b-4370-b0c9-00da4e9a8a05'

In [32]:
for n, i in tqdm(enumerate(dcms), total=len(dcms)):
    bn = os.path.basename(i)
    n = os.path.splitext(bn)[0]
    png_n = n+'.png'
    png_path = PNGS_TEST/png_n
    from_dicom_to_png(str(i), png_path)